In [29]:
import torch
import os
import pickle
import PIL
path = os.Path()
model = torch.load(path/'model_convnext_tiny_unfreeze.pkl', cpu=True, pickle_module=pickle)


In [30]:
from flask import Flask, flash, request, redirect, url_for, render_template
import os
from werkzeug.utils import secure_filename

import numpy as np
import itertools as IT


In [31]:
app = Flask(__name__)
app.secret_key = "secret"
app.config['UPLOAD_FOLDER'] = 'static/uploads/'

In [32]:
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg'])
 
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [33]:
@app.route('/')
def index():
    return render_template('index.html')
 
@app.route('/predict')
def predict():
    return render_template('predict.html')

@app.route('/predict', methods=['POST'])
def upload_image():
    if 'files[]' not in request.files:
        flash('No file part')
        return redirect(request.url)
    
    files = request.files.getlist('files[]')
    file_names = []
    pred_results = []
    for file in files:
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file_names.append(filename)

            # print(__file__)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            
            image = PIL.PILImage.create(app.config['UPLOAD_FOLDER'] + filename)

            pred, pred_idx, probs = model.predict(image)
            prediction = f'{pred}; Probability: {probs[pred_idx]:.04f}'
            pred_results.append(prediction)
        else:
            flash('Allowed image types are -> png, jpg, jpeg, gif')
            return redirect(request.url)
 
    return render_template('predict.html', filenames=file_names, predictions=pred_results, zip=zip)
 
@app.route('/display/<filename>')
def display_image(filename):
    return redirect(url_for('static', filename='uploads/' + filename), code=301)



In [34]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [04/Oct/2022 17:05:03] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2022 17:05:03] "GET /static/styles/style.css HTTP/1.1" 304 -


127.0.0.1 - - [04/Oct/2022 17:05:10] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2022 17:05:10] "GET /static/styles/style.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Oct/2022 17:05:10] "GET /display/100083.jpg HTTP/1.1" 301 -
127.0.0.1 - - [04/Oct/2022 17:05:10] "GET /static/uploads/100083.jpg HTTP/1.1" 200 -
